Lets train a keras model for segmentation and see its fps on CPU ! 

In [1]:
import os, logging

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
logging.getLogger("tensorflow").setLevel(logging.CRITICAL)
logging.getLogger("tensorflow_hub").setLevel(logging.CRITICAL)


from utils import (data_path, Display, get_train_data, DisplayCallback, DataGenerator)
import tensorflow as tf
from common import lr_schedule, get_segmentation_model

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [2]:
from train_segmentation import train

In [3]:
big_model_path = "demodir/" 
saved_big_model_path = "demodir/saved_model/"
tflite_model_path = "demodir/quantized.tflite"
video_path = "../yogaposes/videos/2019-10-10-183635.webm"

In [4]:
model = train(big_model_path)


Epoch 00001: LearningRateScheduler reducing learning rate to 0.0002.
321/322 [============================>.] - ETA: 0s - loss: 0.7417 - iou_score: 0.6858
Epoch 00001: val_loss improved from inf to 0.62214, saving model to demodir//weights.hdf5
322/322 [==============================] - 105s 325ms/step - loss: 0.7408 - iou_score: 0.6867 - val_loss: 0.6221 - val_iou_score: 0.6514


In [5]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

In [6]:
!du -sh demodir/saved_model/

62M	demodir/saved_model/


In [7]:
from compress import compress

In [8]:
compress(saved_big_model_path, tflite_model_path, img_size=128)

9726100


In [9]:
!du -sh demodir/quantized.tflite

9.3M	demodir/quantized.tflite


In [10]:
from segmentation_speed import compare_speed

In [15]:
compare_speed(big_model_path + "weights.hdf5", tflite_model_path, video_path)

Running model ... : 100%|██████████| 200/200 [00:11<00:00, 17.13it/s]
